In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import copy

os.chdir("C:\\Users\\Hunsoo\\Desktop\\regression_practice")

#dataset_path = os.path.join(os.getcwd(),"Data")
#dataset_path

In [2]:
### 기상데이터 인덱스에 맞게 영상 불러오기 ###

In [3]:
raw_dataset = pd.read_csv('YA_2015_2017_data_GMT.txt',sep="\t", header=0)
raw_dataset.head(20)

,point,year,month,day,day365,clock,GMT,temp,windspeed,winddirection,...,pv_4h,pv_3h,pv_2h,pv_1h,pv0h,pv1h,pv2h,pv3h,pv4h,pv5h
0,165,2015,1,1,1,0,15,-1.2,7.6,NaN,...,0,0,0,0,0,0,0,0,0,0
1,165,2015,1,1,1,1,16,-1.8,10.4,NaN,...,0,0,0,0,0,0,0,0,0,0
2,165,2015,1,1,1,2,17,-1.7,10.9,NaN,...,0,0,0,0,0,0,0,0,0,0
3,165,2015,1,1,1,3,18,-1.6,11.6,NaN,...,0,0,0,0,0,0,0,0,0,5
4,165,2015,1,1,1,4,19,-1.8,10.8,NaN,...,0,0,0,0,0,0,0,0,5,139
5,165,2015,1,1,1,5,20,-1.6,9.7,340.0,...,0,0,0,0,0,0,0,5,139,732
6,165,2015,1,1,1,6,21,-2.3,7.2,340.0,...,0,0,0,0,0,0,5,139,732,590
7,165,2015,1,1,1,7,22,-2.7,8.3,NaN,...,0,0,0,0,0,5,139,732,590,787
8,165,2015,1,1,1,8,23,-2.7,10.5,NaN,...,0,0,0,0,5,139,732,590,787,327
9,165,2015,1,1,1,9,0,-2.0,9.7,NaN,...,0,0,0,5,139,732,590,787,327,588


In [5]:
### 월,일 앞에 0 붙여주기 ###

raw_dataset['month'] = raw_dataset['month'].apply(lambda x: '{0:0>2}'.format(x))

raw_dataset['day'] = raw_dataset['day'].apply(lambda x: '{0:0>2}'.format(x))

raw_dataset['clock'] = raw_dataset['clock'].apply(lambda x: '{0:0>2}'.format(x))

raw_dataset['GMT'] = raw_dataset['GMT'].apply(lambda x: '{0:0>2}'.format(x))

In [8]:
### 영상담을 공간 만들기 ###

coms_data_ir1 = []
coms_data_ir2 = []
coms_data_wv = []
coms_data_swir = []
coms_data_vis = []

In [9]:
# 발전소 위치 #
pv_row=682
pv_col=503

In [ ]:
### 'YA_2015_2017_data_GMT.txt' 기상데이터에 맞춰서, 영상 읽은후, 발전소 위치 픽셀 값 저장하는거 ###


for i in range(len(raw_dataset)):
    year_i=raw_dataset.year[i]
    month_i=raw_dataset.month[i]
    day_i=raw_dataset.day[i]
    GMT_i=raw_dataset.GMT[i]
    path_name = 'E:\\COMS\\'+'Y'+str(year_i)+'\\'+'M'+str(month_i)+'\\'+'D'+str(day_i)+'\\'
    os.chdir(path_name)
    #print(os.listdir(path_name))
    
    # ir1
    file_name = 'coms_le1b_ir1_ch1_k_'+str(year_i)+str(month_i)+str(day_i)+str(GMT_i)+'00.bin'
    if os.path.exists(file_name):
        f = open(file_name,'rb')
        bdata = np.fromfile(f,dtype=">u2",count=-1)
        bdata_np_reshape = np.reshape(bdata,[1024,1024])
        #plt.imshow(bdata_np_reshape)
        #plt.figure()
        #print(bdata_np_reshape[500,500])
        coms_data_ir1.append(bdata_np_reshape[pv_row,pv_col])
    else:
        print("skip")
        coms_data_ir1.append("skip")
    
    # ir2
    file_name = 'coms_le1b_ir2_ch2_k_'+str(year_i)+str(month_i)+str(day_i)+str(GMT_i)+'00.bin'
    if os.path.exists(file_name):
        f = open(file_name,'rb')
        bdata = np.fromfile(f,dtype=">u2",count=-1)
        bdata_np_reshape = np.reshape(bdata,[1024,1024])
        #plt.imshow(bdata_np_reshape)
        #plt.figure()
        #print(bdata_np_reshape[500,500])
        coms_data_ir2.append(bdata_np_reshape[pv_row,pv_col])
    else:
        print("skip")
        coms_data_ir2.append("skip")
        

    # wv
    file_name = 'coms_le1b_wv_ch3_k_'+str(year_i)+str(month_i)+str(day_i)+str(GMT_i)+'00.bin'
    if os.path.exists(file_name):
        f = open(file_name,'rb')
        bdata = np.fromfile(f,dtype=">u2",count=-1)
        bdata_np_reshape = np.reshape(bdata,[1024,1024])
        #plt.imshow(bdata_np_reshape)
        #plt.figure()
        #print(bdata_np_reshape[500,500])
        coms_data_wv.append(bdata_np_reshape[pv_row,pv_col])
    else:
        print("skip")
        coms_data_wv.append("skip")
        
    # swir
    file_name = 'coms_le1b_swir_ch4_k_'+str(year_i)+str(month_i)+str(day_i)+str(GMT_i)+'00.bin'
    if os.path.exists(file_name):
        f = open(file_name,'rb')
        bdata = np.fromfile(f,dtype=">u2",count=-1)
        bdata_np_reshape = np.reshape(bdata,[1024,1024])
        #plt.imshow(bdata_np_reshape)
        #plt.figure()
        #print(bdata_np_reshape[500,500])
        coms_data_swir.append(bdata_np_reshape[pv_row,pv_col])
    else:
        print("skip")
        coms_data_swir.append("skip")
        
    # vis
    file_name = 'coms_le1b_vis_ch5_k_'+str(year_i)+str(month_i)+str(day_i)+str(GMT_i)+'00.bin'
    if os.path.exists(file_name):
        f = open(file_name,'rb')
        bdata = np.fromfile(f,dtype=">u2",count=-1)
        bdata_np_reshape = np.reshape(bdata,[1024,1024])
        #plt.imshow(bdata_np_reshape)
        #plt.figure()
        #print(bdata_np_reshape[500,500])
        coms_data_vis.append(bdata_np_reshape[pv_row,pv_col])
    else:
        print("skip")
        coms_data_vis.append("skip")
    

skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip


In [ ]:
### 픽셀 칼럼 따로 만들어주고, 데이터 넣어주기 ###

raw_dataset['coms_data_ir1']=coms_data_ir1
raw_dataset['coms_data_ir2']=coms_data_ir2
raw_dataset['coms_data_wv']=coms_data_wv
raw_dataset['coms_data_swir']=coms_data_swir
raw_dataset['coms_data_vis']=coms_data_vis

In [ ]:
### 불러오는데 너무 오래걸리니깐 따로 저장해두기 ###

raw_dataset_with_coms_pix= copy.deepcopy(raw_dataset)

os.chdir("C:\\Users\\Hunsoo\\Desktop\\regression_practice")
raw_dataset_with_coms_pix.to_csv("raw_dataset_with_coms_pix.csv", mode='w
                                 

##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  ##  
## 엑셀 가서 Multi Temporal로 바꿔서 저장해줘야됨 ######@@@@@@@@@@@@@@@
